In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.externals import joblib
from skrvm import RVR,RVC
import time

/home/veruska_santos/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
data_path = '../data/normalized_data_X_headway_5p.csv'
y_path = '../data/y_headway_5p.csv'

data = pd.read_csv(data_path)
y = pd.read_csv(y_path)
print(data.shape)
print(y.shape)
data.head()

(69884, 420)
(69884, 2)


,route,tripNum,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,...,alertTypeSB_ACCIDENT,alertTypeSB_CHIT_CHAT,alertTypeSB_HAZARD,alertTypeSB_JAM,alertTypeSB_NORMAL,alertTypeSB_POLICE,alertTypeSB_ROAD_CLOSED,jamBlockTypeSB_-,jamBlockTypeSB_NORMAL,jamBlockTypeSB_ROAD_CLOSED_EVENT
0,0.118068,0.0,0.084945,0.264128,0.439400,0.010487,0.0,0.000006,0.265656,0.439493,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.118068,0.0,0.084965,0.264428,0.443488,0.014304,0.0,0.000009,0.265956,0.444051,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.118068,0.0,0.084972,0.263625,0.448606,0.018030,0.0,0.000010,0.264915,0.448957,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.118068,0.0,0.084979,0.261608,0.458297,0.025182,0.0,0.000014,0.263352,0.456961,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.118068,0.0,0.084985,0.260228,0.463266,0.028953,0.0,0.000000,0.261792,0.463170,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [3]:
data = data[0:10000]
y = y[0:10000]
print(data.shape)
print(y.shape)

(10000, 420)
(10000, 2)


In [3]:
data.tripNum.unique()

array([0.        , 0.06666667, 0.03333333, 0.1       , 0.13333333,
       0.16666667, 0.2       , 0.23333333, 0.26666667, 0.3       ,
       0.33333333, 0.36666667, 0.4       , 0.43333333, 0.46666667,
       0.5       , 0.53333333, 0.56666667, 0.63333333, 0.66666667,
       0.6       , 0.7       , 0.73333333, 0.8       , 0.83333333,
       0.86666667])

In [4]:
y

,headway,headwayThreshold
0,143.0,5.0
1,143.0,5.0
2,143.0,5.0
3,143.0,5.0
4,143.0,5.0
...,...,...
69879,0.0,2.0
69880,9.0,5.0
69881,18.0,5.0
69882,41.0,5.0


In [3]:
data['busBunching'] = y
# del data
# del y

In [ ]:
# filtering one route: data
# 1 day (5p): 69884 data


#hd_alto.groupby(['busCode'])['busCode'].nunique().count())
data.groupby(['route', 'busBunching'])['route', 'busBunching'].count()


In [4]:
# Making training and test data: 80% Training, 20% Test
random.seed(15) #to get always the same set
train_X, test_X, train_Y, test_Y = train_test_split(data, y.headway, test_size=0.20, random_state=7)

### RVM

In [5]:
model = RVR(kernel='poly')
model.fit(train_X, train_Y)

/home/veruska_santos/.local/lib/python3.6/site-packages/skrvm/rvm.py:153: RuntimeWarning: divide by zero encountered in true_divide
  self.alpha_ = self.gamma/(self.m_ ** 2)
/home/veruska_santos/.local/lib/python3.6/site-packages/skrvm/rvm.py:108: RuntimeWarning: invalid value encountered in less
  keep_alpha = self.alpha_ < self.threshold_alpha


RVR(alpha=1e-06, beta=1e-06, beta_fixed=False, bias_used=True, coef0=0.0,
    coef1=None, degree=3, kernel='poly', n_iter=3000,
    threshold_alpha=1000000000.0, tol=0.001, verbose=False)

In [6]:
# y_test_path = '../../data/output/test_Y.csv'
# x_test_path = '../../data/output/test_X.csv'

# test_X = pd.read_csv(x_test_path)
# test_Y = pd.read_csv(y_test_path)

pred_array = model.predict(test_X)
pred_array

array([nan, nan, nan, ..., nan, nan, nan])

In [7]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #30000 - rbf

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [19]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #4000 - linear

RMSE: 21.65898592649296


In [7]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #30000 - linear

RMSE: 44.20423303662642


In [7]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #10000 - linear

RMSE: 42.61607032953796


In [16]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #4000 - linear

RMSE: 21.640988155749465


In [7]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #3000 - rbm

RMSE: 25.541931960428716


In [10]:
rmse = np.sqrt(mean_squared_error(test_Y, pred_array))
print('RMSE: ' + str(rmse)) #1000 - rbm

RMSE: 45.571023004277336
